# Simulación mínima con FLAME GPU y proximidad entre países

Este cuaderno arma un ejemplo muy reducido que toma algunas relaciones de proximidad entre países a partir de `data/country_proximity_edges.csv` y ejecuta una simulación probabilística de contagio usando FLAME GPU. La idea es solamente comprobar que la tubería funciona de punta a punta con un conjunto pequeño de agentes.


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

CUDA_BASE = r"C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v13.0"
for extra_path in (
    fr"{CUDA_BASE}\bin",
    fr"{CUDA_BASE}\nvvm\bin",
):
    if os.path.exists(extra_path):
        os.add_dll_directory(extra_path)

os.environ.setdefault("FLAMEGPU_LOG_LEVEL", "info")

import pyflamegpu as fg
print("pyflamegpu:", fg.__version__)


In [ ]:
edges_path = "data/country_proximity_edges.csv"
edges_df = pd.read_csv(edges_path)

subset_countries = [
    "Argentina",
    "Bolivia",
    "Brazil",
    "Chile",
    "Paraguay",
    "Uruguay",
]

subset_edges = edges_df[
    edges_df["Pais_A"].isin(subset_countries)
    & edges_df["Pais_B"].isin(subset_countries)
].copy()
print(f"Aristas filtradas: {len(subset_edges)}")

country_to_id = {name: idx for idx, name in enumerate(subset_countries)}
num_agents = len(subset_countries)
proximity_matrix = np.zeros((num_agents, num_agents), dtype=np.float32)

for row in subset_edges.itertuples(index=False):
    i = country_to_id[row.Pais_A]
    j = country_to_id[row.Pais_B]
    value = float(row.Proximidad)
    if value > proximity_matrix[i, j]:
        proximity_matrix[i, j] = value
    if value > proximity_matrix[j, i]:
        proximity_matrix[j, i] = value

np.fill_diagonal(proximity_matrix, 0.0)

proximity_df = pd.DataFrame(
    proximity_matrix,
    index=subset_countries,
    columns=subset_countries,
)
proximity_df.round(3)


In [ ]:
base_transmission = 0.35
proximity_list = proximity_matrix.flatten().astype(np.float32).tolist()

model = fg.ModelDescription("MiniCountrySpread")

country_agent = model.newAgent("Country")
country_agent.newVariableUInt("country_id")
country_agent.newVariableInt("infected")

message = model.newMessageBruteForce("InfectedMsg")
message.newVariableUInt("id")

spread_func = country_agent.newRTCFunction(
    "output_infection",
    """
    FLAMEGPU_AGENT_FUNCTION(output_infection, flamegpu::MessageNone, flamegpu::MessageBruteForce) {
        if (FLAMEGPU->getVariable<int>("infected") == 0) {
            return flamegpu::ALIVE;
        }
        const unsigned int id = FLAMEGPU->getVariable<unsigned int>("country_id");
        FLAMEGPU->message_out.setVariable<unsigned int>("id", id);
        return flamegpu::ALIVE;
    }
    """
)
spread_func.setMessageOutput("InfectedMsg")

receive_func = country_agent.newRTCFunction(
    "receive_infection",
    """
    FLAMEGPU_AGENT_FUNCTION(receive_infection, flamegpu::MessageBruteForce, flamegpu::MessageNone) {
        if (FLAMEGPU->getVariable<int>("infected") == 1) {
            return flamegpu::ALIVE;
        }
        const unsigned int self_id = FLAMEGPU->getVariable<unsigned int>("country_id");
        const unsigned int num_agents = FLAMEGPU->environment.getProperty<unsigned int>("num_agents");
        const float base_rate = FLAMEGPU->environment.getProperty<float>("base_transmission");
        for (const auto &msg : FLAMEGPU->message_in) {
            const unsigned int other_id = msg.getVariable<unsigned int>("id");
            if (other_id == self_id) {
                continue;
            }
            const float proximity = FLAMEGPU->environment.getProperty<float>("proximity_matrix", self_id * num_agents + other_id);
            if (proximity <= 0.0f) {
                continue;
            }
            const float attempt = base_rate * proximity;
            if (attempt <= 0.0f) {
                continue;
            }
            const float r = FLAMEGPU->random.uniform<float>();
            if (r < attempt) {
                FLAMEGPU->setVariable<int>("infected", 1);
                break;
            }
        }
        return flamegpu::ALIVE;
    }
    """
)
receive_func.setMessageInput("InfectedMsg")

layer_out = model.newLayer("spread")
layer_out.addAgentFunction(spread_func)
layer_in = model.newLayer("receive")
layer_in.addAgentFunction(receive_func)

env = model.Environment()
env.newPropertyUInt("num_agents", num_agents)
env.newPropertyFloat("base_transmission", base_transmission)
env.newPropertyArrayFloat("proximity_matrix", proximity_list)

print("Modelo creado con", num_agents, "agentes")


In [ ]:
initial_infected = {"Argentina"}

init_population = fg.AgentVector(country_agent, num_agents)
for idx, country in enumerate(subset_countries):
    agent = init_population[idx]
    agent.setVariableUInt("country_id", idx)
    agent.setVariableInt("infected", 1 if country in initial_infected else 0)

simulation = fg.CUDASimulation(model)
simulation.SimulationConfig().random_seed = 2025

simulation.setPopulationData(init_population)
simulation.setEnvironmentPropertyUInt("num_agents", num_agents)
simulation.setEnvironmentPropertyFloat("base_transmission", base_transmission)
simulation.setEnvironmentPropertyArrayFloat("proximity_matrix", proximity_list)

print("Agentes inicializados. Infectados al inicio:")
for country, idx in country_to_id.items():
    status = "infectado" if country in initial_infected else "susceptible"
    print(f"- {country}: {status}")


In [ ]:
def snapshot(simulation_obj):
    agent_vec = fg.AgentVector(country_agent)
    simulation_obj.getPopulationData(agent_vec)
    records = []
    for agent in agent_vec:
        idx = agent.getVariableUInt("country_id")
        records.append(
            {
                "id": int(idx),
                "country": subset_countries[idx],
                "infected": int(agent.getVariableInt("infected")),
            }
        )
    return records

print("Función de snapshot lista")


In [ ]:
steps = 8
history = []
per_agent_records = []

initial_state = snapshot(simulation)
for rec in initial_state:
    rec = rec.copy()
    rec["step"] = 0
    per_agent_records.append(rec)
history.append({"step": 0, "infected": sum(r["infected"] for r in initial_state)})

for step in range(1, steps + 1):
    simulation.step()
    state = snapshot(simulation)
    for rec in state:
        new_rec = rec.copy()
        new_rec["step"] = step
        per_agent_records.append(new_rec)
    history.append({"step": step, "infected": sum(r["infected"] for r in state)})

history_df = pd.DataFrame(history)
agent_df = pd.DataFrame(per_agent_records)
history_df


In [ ]:
status_table = (
    agent_df.pivot_table(index="country", columns="step", values="infected", aggfunc="max")
    .astype(int)
    .sort_index()
)
status_table


In [ ]:
plt.figure(figsize=(6, 3.5))
plt.plot(history_df["step"], history_df["infected"], marker="o")
plt.title("Conteo de países infectados")
plt.xlabel("Paso")
plt.ylabel("Países infectados")
plt.ylim(0, num_agents + 0.5)
plt.grid(alpha=0.3)
plt.show()


## Qué observar
- El modelo usa solo seis países del Cono Sur para mantener el tamaño reducido.
- La probabilidad de contagio se calcula combinando un parámetro global (`base_transmission`) con la proximidad normalizada del CSV.
- Podés cambiar los países iniciales en `initial_infected` o ajustar `base_transmission` para experimentar rápidamente.
- Si necesitás registrar más pasos, modificá la variable `steps` y volvés a ejecutar las celdas de simulación.
